# Ejercicio 3 - Corrección de errores cuántica

## Contexto histórico

El algoritmo de Shor les otorgó a las computadoras cuánticas una utilidad práctica, pero el ruido inherente de la mecánica cuántica pone en evidencia que construir un hardware capaz de ejecutar tal algoritmo es un gran desafío. En 1995, Shor estrenó otra publicación fundamental: un esquema que compartía la información en múltiples cúbits con la intención de reducir los errores.[1]

Grandes progresos se han logrado en las décadas transcurridas desde entonces. Nuevos tipos de códigos de corrección de errores se han descubierto, y un gran marco teórico se ha construido en torno a estos. Los códigos de superficie (del inglés 'surface code') propuestos por Kitaev en 1997 han emergido como los candidatos más destacados, y a partir del diseño original muchas versiones han emergido desde entonces. Pero todavía hay mucho progreso por hacer en la confección de códigos que concuerden con los detalles específicos del hardware cuántico a disposición.[2]

En este ejercicio, consideraremos un caso en cual 'errores' artificiales se insertan en un circuito. Tu tarea es la de diseñar el circuito de manera que estas compuertas adicionales puedan ser identificadas.

Tendrás que pensar cómo implementar tu circuito en un verdadero dispositivo. Esto significa que tendrás que confeccionar una solución para que concuerde con la configuración de los cúbits en el dispositivo. Tu solución será evaluada en función de cuán pequeño es el número de compuertas entrelazantes (que son las compuertas más ruidosas) que utilices. 


### Referencias
1. Shor, Peter W. "Scheme for reducing decoherence in quantum computer memory." Physical review A 52.4 (1995): R2493.
1. Dennis, Eric, et al. "Topological quantum memory." Journal of Mathematical Physics 43.9 (2002): 4452-4505.

## El problema de los errores

Errores ocurren cuando una operación indeseada actúa sobre nuestros cúbits. Sus efectos hacen que las cosas salgan mal en nuestros circuitos. Los resultados extraños que has podido ver cuando se ejecuta un programa en un verdadero dispositivo siempre son debidos a estos errores.

Existen muchas operaciones indeseadas que pueden ocurrir, pero resulta que podemos pretender que solo hay dos tipos de error: inversiones de bits o inversiones de fase.

Inversiones de bits tienen el mismo efecto que la compuerta `x`. Invierte el estado $|0\rangle$ de un cúbit al estado $|1\rangle$ y viceversa. Inversiones de fase tienen el mismo efecto que la la compuerta `z`, introducen una fase de $-1$ a las superposiciones. Sencillamente, invierten el estado $|+\rangle$ de un cúbit al estado $|-\rangle$ y viceversa.

La razón por la que podemos pensar en cualquier error en términos de estas dos se debe a que cualquier error puede representarse por una matriz, y cualquier matriz puede escribirse en términos de matrices $X$ y $Z$. Específicamente, para cualquier matriz $M$ de un solo cúbit, 

$$
M = \alpha I + \beta X + \gamma XZ + \delta Z,
$$
con los valores de $\alpha$, $\beta$, $\gamma$ y $\delta$ apropiados.

Así que siempre que apliquemos esta matriz a algún estado a un solo cúbit $|\psi\rangle$ obtendremos

$$
M |\psi\rangle = \alpha |\psi\rangle + \beta X |\psi\rangle + \gamma XZ |\psi\rangle + \delta Z |\psi\rangle.
$$

La superposición resultante está compuesta del estado original, el estado que obtendríamos si el error es solamente una inversión del bit, el estado después de una inversión de la fase solamente y el estado después de aplicar ambas. Si tuviésemos una manera de medir si tuvo lugar la inversión de bit o la inversión de fase, el estado colapsaría a una sola posibilidad. Y nuestro error complejo se convertiría en una inversión simple de bit o de fase.

Así que ¿cómo detectamos si hubo una inversión de bit o una inversión de fase (o ambas)? ¿Y qué hacemos al respecto una vez que sepamos? Responder estas preguntas es de lo que se trata la corrección de errores cuántica.

## Un ejemplo excesivamente sencillo
Uno de los circuitos cuánticos que la mayoría de las personas escriben por primera vez consiste en crear un par de cúbits entrelazados. En este viaje a través de la corrección de errores cuántica, comenzaremos de la misma manera.

In [ ]:
from qiskit import QuantumCircuit, Aer

# Crea un estado entrelazado
qc_init = QuantumCircuit(2)
qc_init.h(0)
qc_init.cx(0,1)

# Dibuja el circuito
display(qc_init.draw('mpl'))

# Se obtiene la salida
qc = qc_init.copy()
qc.measure_all()
job = Aer.get_backend('qasm_simulator').run(qc)
job.result().get_counts()

Aquí vemos el resultado esperado cuando ejecutamos el circuito: los resultados `00` y `11` ocurren con probabilidades iguales.

Pero qué pasa cuando tenemos el mismo circuito, pero se inserta manualmente un 'error' que invierte el bit.

In [ ]:
# Hace la inversión del bit
qc_insert = QuantumCircuit(2)
qc_insert.x(0)

# Se añade al circuito original
qc = qc_init.copy()
qc = qc.compose(qc_insert)

# Dibuja el circuito
display(qc.draw('mpl'))

# Se obtiene la salida
qc.measure_all()
job = Aer.get_backend('qasm_simulator').run(qc)
job.result().get_counts()

Ahora los resultados son diferentes `01` y `10`. Los dos valores de los bits han pasado de estar siempre de acuerdo, a siempre en desacuerdo. De esta manera, detectamos el efecto del error.

Otra manera de detectarlo es deshacer el entrelazamiento con unas pocas compuertas adicionales. Si no hay errores, regresamos al estado inicial $|00\rangle$.

In [ ]:
# Deshace el entrelazamiento
qc_syn = QuantumCircuit(2)
qc_syn.cx(0,1)
qc_syn.h(0)

# Se añade después del error
qc = qc_init.copy()
qc = qc.compose(qc_syn)

# Dibuja el circuito
display(qc.draw('mpl'))

# Se obtiene la salida
qc.measure_all()
job = Aer.get_backend('qasm_simulator').run(qc)
job.result().get_counts()

Pero ¿qué pasa si hay un error en uno de los cúbits? Intenta insertar diferentes errores para averiguarlo.

Aquí está un circuito con todos los componentes que introducimos hasta ahora: la preparación `qc_init`, la inserción del error `qc_insert`, y la parte final `qc_syn` que asegura que la medida final dé una respuesta definitiva.


In [ ]:
# Define un error
qc_insert = QuantumCircuit(2)
qc_insert.x(0)

# Deshace el entrelazamiento
qc_syn = QuantumCircuit(2)
qc_syn.cx(0,1)
qc_syn.h(0)

# Se añade después del error
qc = qc_init.copy()
qc = qc.compose(qc_insert)
qc = qc.compose(qc_syn)

# Dibuja el circuito
display(qc.draw('mpl'))

# Se obtiene la salida
qc.measure_all()
job = Aer.get_backend('qasm_simulator').run(qc)
job.result().get_counts()

Encontrarás que la salida nos dice exactamente qué sucede con los errores. Ambas, la inversión de bit y la inversión de fase pueden ser detectadas. El valor del bit de la izquierda es `1` si y solo si hay una inversión de bit (como si hubiésemos introducido un `x(0)` o un `x(1)`). De manera similar, el bit de la derecha nos dice si hubo una inversión de fase (si se introdujo un `z(0)` o un `z(1)`).

Esta capacidad de detectar y distinguir inversiones de bit o de fase es muy útil. Pero no es lo suficientemente útil. Solo podemos decir *qué tipo* de errores están ocurriendo, pero no *dónde*. Sin más detalles, no es posible averiguar cómo eliminar los efectos de estas operaciones de nuestros cálculos. Para hacer la corrección de errores cuántica necesitamos por lo tanto algo mejor y más grande.  

¡Esa es justamente tu tarea! Aquí hay una lista de lo que necesitas enviar como respuesta. Todo esto será explicado en el ejemplo que sigue. 


<div class="alert alert-block alert-success">

<b>Objetivo</b>

Crear un circuito que puede detectar errores `x` y `z` sobre dos cúbits. Puedes idear tu propia solución. O simplemente retoca la solución casi válida presentada a continuación. 

</div>

<div class="alert alert-block alert-danger">
<b>Qué hay que enviar</b> 

* Necesitas suministrar dos circuitos:
    * `qc_init`: Prepara los cúbits (en los cuáles hay al menos dos) en un estado inicial de preferencia
    * `qc_syn`: Mide un subconjunto de cúbits.

* Los errores artificiales a insertar son compuertas `x` y `z` en dos cúbits específicos. Debes escoger estos dos cúbits (suministrarlos en una lista llamada `error_qubits`).

* Hay 16 posibles series de errores que se pueden añadir (incluido el caso trivial de cero errores). La medida resultante de `qc_syn` deberá resultar en un único código binario para cada serie. El evaluador te devolverá un mensaje de error en inglés *'Please make sure the circuit is created to the initial layout.'* (*Por favor asegúrate que el circuito fue creado para la configuración inicial*) si está condición no se satisface.

* El evaluador compilará el circuito total en el backend `ibmq_tokyo` (un dispositivo que ha sido desactivado). Para poder demostrar que tu solución está confeccionada a la medida del dispositivo, esta transpilación no debería cambiar el número de compuertas `cx`. Si es el caso, obtendrás el mensaje de error en inglés *'Please make sure the circuit is created to the initial layout.'* (*Por favor asegúrate que el circuito fue creado para la configuración inicial*).
    
* Para orientar la transpilación, deberás decirle al transpilador cuáles cúbits del dispositivo deberán ser usados en tu circuito. Esto se puede llevar a cabo con la lista `initial_layout`.
    
* Puedes comenzar con el ejemplo a continuación, que puede convertirse en una respuesta válida con unos pocos retoques.
</div>

## Un mejor ejemplo: el código de superficie

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, transpile

import qiskit.tools.jupyter
from qiskit.test.mock import FakeTokyo

En este ejemplo, usaremos 5 cúbits para el código que llamaremos `code`. Para poder seguirlos, definiremos un registro cuántico especial

In [ ]:
code = QuantumRegister(5,'code')

También usaremos cuatro cúbits addicionales que llamaremos cúbits del síndrome o `syn`.

In [ ]:
syn = QuantumRegister(4,'syn')

De manera similar, definimos un registro `out` de cuatro cúbits para la salida, que utilizaremos para medir los cúbits del síndrome.

In [ ]:
out = ClassicalRegister(4,'output')

Consideraremos que los cúbits están dispuestos de la siguiente manera, con los cúbits del código formando las esquinas de cuatro triangulos, y los cúbits del síndrome se ubican en el interior de cada triángulo

```
c0----------c1
| \   s0   / |
|   \    /   |
| s1  c2  s2 |
|   /    \   |
| /   s3   \ |
c3----------c4
```

Para cada triángulo, asociamos una operación estabilizadora sobre cada uno de los tres cúbits. Para los cúbits de los lados, los estabilizadores son ZZZ. Para el cúbit superior e inferior, los estabilizadores son XXX.

La medida del circuito del síndrome, corresponde a la medida de estos observables. Esto se lleva a cabo de manera similar con los estabilizadores del código de superficie (de hecho, este código es una versión reducida de un código de superficie).

<div class="alert alert-block alert-danger">
 
<b>Advertencia</b> 

Debes eliminar las barreras `barrier()` antes de enviar el código, ya que pueden interferir con la transpilación. Las añadimos aquí para permitir una visualización más directa.
</div>

In [ ]:
qc_syn = QuantumCircuit(code,syn,out)


# ZZZ de la izquierda
qc_syn.cx(code[0],syn[1])
qc_syn.cx(code[2],syn[1])
qc_syn.cx(code[3],syn[1])
qc_syn.barrier()

# ZZZ de la derecha
qc_syn.cx(code[1],syn[2])
qc_syn.cx(code[2],syn[2])
qc_syn.cx(code[4],syn[2])
qc_syn.barrier()

# XXX superior
qc_syn.h(syn[0])
qc_syn.cx(syn[0],code[0])
qc_syn.cx(syn[0],code[1])
qc_syn.cx(syn[0],code[2])
qc_syn.h(syn[0])
qc_syn.barrier()

# XXX inferior
qc_syn.h(syn[3])
qc_syn.cx(syn[3],code[2])
qc_syn.cx(syn[3],code[3])
qc_syn.cx(syn[3],code[4])
qc_syn.h(syn[3])
qc_syn.barrier()


# Se miden los cúbits auxiliares
qc_syn.measure(syn,out)
qc_syn.draw('mpl')

La preparación del circuito inicializa un autoestado de estos observables, de manera que la salida de la medida del síndrome resultará en `0000` con toda certeza.

In [ ]:
qc_init = QuantumCircuit(code,syn,out)

qc_init.h(syn[0])
qc_init.cx(syn[0],code[0])
qc_init.cx(syn[0],code[1])
qc_init.cx(syn[0],code[2])
qc_init.cx(code[2],syn[0])

qc_init.h(syn[3])
qc_init.cx(syn[3],code[2])
qc_init.cx(syn[3],code[3])
qc_init.cx(syn[3],code[4])
qc_init.cx(code[4],syn[3])

qc_init.barrier()
qc_init.draw('mpl')

Verifiquemos que esto es cierto.

In [ ]:
qc = qc_init.compose(qc_syn)
display(qc.draw('mpl'))

job = Aer.get_backend('qasm_simulator').run(qc)
job.result().get_counts()

Ahora hagamos un circuito que puede añadir compuertas x` y `z` sobre nuestro par de cúbits del código. Para ello, necesitamos escoger cuáles de estos 5 cúbits del código corresponden a los dos cúbits requeridos para validar esta condición.

Para este código tenemos que escoger esquinas opuestas.

In [ ]:
error_qubits = [0,4]

Aquí 0 y 4 se refieren a las posiciones de los cúbits en la lista siguiente, y por lo tanto son justamente los cúbits  `code[0]` y `code[4]`.


In [ ]:
qc.qubits

Para verificar que el código sigue los requisitos, podemos usar la siguiente función para crear circuitos que introducen errores artificiales. Aquí los errores que queremos añadir están enumerados en `errors` como una simple cadena de texto, por ejemplo `x0` para `x` aplicado sobre `error_qubits[0]`.

In [ ]:
def insert(errors,error_qubits,code,syn,out):

    qc_insert = QuantumCircuit(code,syn,out)

    if 'x0' in errors:
        qc_insert.x(error_qubits[0])
    if 'x1' in errors:
        qc_insert.x(error_qubits[1])
    if 'z0' in errors:
        qc_insert.z(error_qubits[0])
    if 'z1' in errors:
        qc_insert.z(error_qubits[1])
        
    return qc_insert

En vez mirar todas las 16 posibilidades, concentrémonos en los cuatros casos en los que añadimos un solo error.

In [ ]:
for error in ['x0','x1','z0','z1']:
    
    qc = qc_init.compose(insert([error],error_qubits,code,syn,out)).compose(qc_syn)
    job = Aer.get_backend('qasm_simulator').run(qc)
    
    print('\n Para el error '+error+':')
    counts = job.result().get_counts()
    for output in counts:
        print('La salida es',output,'con un número',counts[output],'de intentos.')

Aquí vemos que cada bit de la salida es `1` cuando un error particular ocurre: el primer bit a la izquierda detecta la aplicación de `z` sobre `error_qubits[1]`, el siguiente detecta la aplicación de `x` sobre `error_qubits[1]`, etcétera. 

<div class="alert alert-block alert-danger">
 
<b>Atención</b> 
El orden correcto de la salida es importante para este ejercicio. Por favor, sigue el orden dado a continuación:
1. En la salida, el primer bit a la izquierda, representa una aplicación de `z` sobre `code[1]`.
2. En la salida, el segundo bit, de izquierda a derecha, representa una aplicación de `x` sobre `code[1]`.
3. En la salida, el tercer bit representa una aplicación de `x` sobre `code[0]`.
4. En la salida, el último bit a la derecha representa una aplicación de `z` sobre `code[0]`.
    
</div>

Mientras más errores afectan al circuito, se vuelve más difícil de decir sin ambigüedad cuáles errores ocurrieron. Sin embargo, repitiendo continuamente la lectura del síndrome para obtener más resultados y para analizar los datos a través del desciframiento, es aún posible la determinación de suficientes errores para corregir estos efectos.

Este tipo de consideraciones van más allá de lo que veremos en este desafío. En cambio, nos concentraremos en algo más sencillo, pero igual de importante: mientras menos errores tengas, y mientras más sencillos sean, mejor será la corrección de errores. Para asegurar esto, tu procedimiento de corrección de errores deberá ser confeccionado en acuerdo con la configuración del dipositivo que utilices.

En este desafío, consideraremos el dispositivo `ibmq_tokyo`. Si bien el verdadero dispositivo fue desactivado hace tiempo, todavía existe como un backend simulado. 

In [ ]:
# Por favor utilizar el backend siguiente
backend = FakeTokyo()
backend

Para tener una primera idea de cómo esta diseñado nuestro circuito original, veamos cuántas compuertas a dos cúbits contiene.

In [ ]:
qc = qc_init.compose(qc_syn)
qc = transpile(qc, basis_gates=['u','cx'])
qc.num_nonlocal_gates()

Si tuviésemos que transpilarlo en el backend `ibmq_tokyo`, daría lugar a una redistribución a costa de agregar más compuertas a dos cúbits.

In [ ]:
qc1 = transpile(qc,backend,basis_gates=['u','cx'], optimization_level=3)
qc1.num_nonlocal_gates()

Podemos controlar esto hasta cierto punto mirando cuáles cúbits en el dispositivo sería mejor utilizar para nuestro código. Si miramos cuáles cúbits en el código necesitan estar conectados por compuertas a dos qubits en `qc_syn`, encontramos el siguiente grafo de conectividad.


```
c0....s0....c1
:      :     :        
:      :     :
s1....c2....s2
:      :     :
:      :     :
c3....s3....c4
```

Ninguna serie de cúbits de `ibmq_tokyo` puede proveer esto, pero ciertas series como 0,1,2,5,6,7,10,11,12, se acercan. Así que podemos establecer una configuración inicial llamada `initial_layout` para decirle al transpilador que la implemente.



In [ ]:
initial_layout = [0,2,6,10,12,1,5,7,11]

Esta lista le dice al transpilador cuáles cúbits en el dispositivo usar cómo cúbits en el circuito (el orden esta enumerado en `qc.qubits`). Así las primeras cinco entradas en esta lista le dicen al circuito cuáles cúbits utilizar cómo cúbits del código y de manera similar las siguientes cuatro entradas en esta lista son para los cúbits del síndrome. Así que utilizamos el cúbit 0 en el dispositivo como `code[0]`, el cúbit 2 como `code[1]`, etc.

Utilicemos esto ahora para la transpilación.

In [ ]:
qc2 = transpile(qc,backend,initial_layout=initial_layout, basis_gates=['u','cx'], optimization_level=3)
qc2.num_nonlocal_gates()

Si bien la transpilación es un proceso aleatorio, típicamente encontrarás que esta usa menos compuertas a dos cúbits, que cuando la configuración inicial no está dada (podrías re-ejecutar la transpilación varias veces para ver que tu código transpilado es un proceso aleatorio). 

No obstante, un esquema de corrección de errores diseñado adecuadamente no necesita hacer ninguna redistribución. Debe ser escrito para el dispositivo exacto a utilizar, y el número de compuertas a dos cúbits debería permanecer constante con certeza. Esta es una condición para que la solución sea válida. Así que no solo tendrás que proveer una configuración en `initial_layout`, sino que también deberás diseñar tus circuitos específicamente para esa configuración.

¡Pero esa parte te la dejamos a ti!

In [ ]:
# Verifica tu respuesta usando este código
from qc_grader import grade_ex3
grade_ex3(qc_init,qc_syn,error_qubits,initial_layout)

In [ ]:
# Envía tu respuesta. Puedes volver a enviarla en todo momento.
from qc_grader import submit_ex3
submit_ex3(qc_init,qc_syn,error_qubits,initial_layout)

## Additional information

**Created by:** James Wootton, Rahul Pratap Singh

**Latin American Spanish translation**: Mauricio Gómez Viloria, Manuel Morgado

**Version:** 1.0.0